In [3]:
import sys
import surprise
import pandas as pd
from utils.timer import Timer
from datasets.python_splitters import python_random_split
from evaluation.python_evaluation import (
    rmse,
    mae,
    rsquared,
    exp_var,
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    get_top_k_items,
)
from models.surprise_utils import (
    predict,
    compute_ranking_predictions,
)
from utils.notebook_utils import store_metadata


print(f"System version: {sys.version}")
print(f"Surprise version: {surprise.__version__}")

System version: 3.11.5 (v3.11.5:cce6ba91b3, Aug 24 2023, 10:50:31) [Clang 13.0.0 (clang-1300.0.29.30)]
Surprise version: 1.1.3


# Explanation
The Singular Value Decomposition (SVD) algorithm used in the Surprise library is a matrix factorization technique commonly employed in recommendation systems. Here's a quick summary of how it works:

- Matrix Factorization: SVD is a technique from linear algebra used to decompose a matrix into three separate matrices. In the context of recommendation systems, it's used to decompose the user-item interaction matrix into two lower-dimensional matrices: one representing users and their latent factors and the other representing items and their latent factors.

- Latent Factors: The idea behind SVD is to represent users and items in a lower-dimensional latent space, where each dimension captures some underlying characteristics or preferences. For example, in a movie recommendation system, latent factors might represent genres, directors, or actors.

- Optimization: The decomposition process aims to find the best approximation of the original matrix by minimizing the error between the predicted ratings and the actual ratings in the training dataset. This optimization is typically achieved using techniques like stochastic gradient descent.

- Prediction: Once the model is trained, it can predict the rating a user would give to an item by taking the dot product of the user's latent factors and the item's latent factors.

- Recommendation: Recommendations can be generated by ranking the predicted ratings for unseen items and recommending the top-ranked items to users.

SVD and its variants have been popular in recommendation systems due to their effectiveness in capturing complex user-item interactions and providing accurate recommendations. The Surprise library provides an easy-to-use interface for implementing SVD and other collaborative filtering algorithms for recommendation tasks.

In [4]:
# Top k items to recommend
TOP_K = 10

In [5]:
# Specify the path to your CSV file
csv_file_path = "/Users/marianareyes/Desktop/ie_tower/chatbots-2/chatlib/datasets/ratings.csv"

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(csv_file_path, usecols=['userID', 'itemID', 'rating'])

data.head()

,userID,itemID,rating
0,1,1,5
1,1,2,3
2,2,3,4
3,3,4,5
4,4,5,4


In [6]:
train, test = python_random_split(data, 0.75)

In [7]:
train_set = surprise.Dataset.load_from_df(
    train, reader=surprise.Reader("ml-100k")
).build_full_trainset()
train_set

In [8]:
svd = surprise.SVD(random_state=0, n_factors=200, n_epochs=30, verbose=True)

with Timer() as train_time:
    svd.fit(train_set)

print(f"Took {train_time.interval} seconds for training.")

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Took 0.01266131000011228 seconds for training.


In [9]:
predictions = predict(svd, test, usercol="userID", itemcol="itemID")
predictions.head()

,userID,itemID,prediction
0,1,1,3.641108
1,5,6,3.797252
2,27,37,3.755556
3,36,46,3.755556
4,4,14,3.781487


In [10]:
with Timer() as test_time:
    all_predictions = compute_ranking_predictions(
        svd, train, usercol="userID", itemcol="itemID", remove_seen=True
    )

print(f"Took {test_time.interval} seconds for prediction.")

Took 0.06732147099683061 seconds for prediction.


In [11]:
all_predictions.head()

,userID,itemID,prediction
45,8,9,3.885442
46,8,7,3.765817
47,8,41,3.983779
48,8,5,3.616633
49,8,44,3.756456


In [12]:
eval_rmse = rmse(test, predictions)
eval_mae = mae(test, predictions)
eval_rsquared = rsquared(test, predictions)
eval_exp_var = exp_var(test, predictions)

eval_map = map_at_k(test, all_predictions, col_prediction="prediction", k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction="prediction", k=TOP_K)
eval_precision = precision_at_k(
    test, all_predictions, col_prediction="prediction", k=TOP_K
)
eval_recall = recall_at_k(test, all_predictions, col_prediction="prediction", k=TOP_K)


print(
    "RMSE:\t\t%f" % eval_rmse,
    "MAE:\t\t%f" % eval_mae,
    "rsquared:\t%f" % eval_rsquared,
    "exp var:\t%f" % eval_exp_var,
    sep="\n",
)

print("----")

print(
    "MAP:\t\t%f" % eval_map,
    "NDCG:\t\t%f" % eval_ndcg,
    "Precision@K:\t%f" % eval_precision,
    "Recall@K:\t%f" % eval_recall,
    sep="\n",
)

RMSE:		1.234595
MAE:		1.138261
rsquared:	-0.051996
exp var:	-0.041246
----
MAP:		0.000000
NDCG:		0.000000
Precision@K:	0.000000
Recall@K:	0.000000


In [13]:
# Specify the user ID for which you want to compute predictions
user_id_to_predict = 1  # Replace 1 with the actual user ID you want to predict for

# Get the items the user has already seen
items_seen_by_user = data[data['userID'] == user_id_to_predict]['itemID'].tolist()

# Filter out the items the user has already seen from all predictions
unseen_items = [item for item in train_set.all_items() if item not in items_seen_by_user]

# Compute predictions for the specified user
user_predictions = [(item, svd.predict(user_id_to_predict, item).est) for item in unseen_items]

# Sort the predictions by rating value in descending order
sorted_predictions = sorted(user_predictions, key=lambda x: x[1], reverse=True)

# Get the top 10 results
top_10_results = sorted_predictions[:10]

# Create a DataFrame for the top 10 results
top_10_df = pd.DataFrame(columns=['userID', 'itemID', 'prediction'])
for item_id, rating in top_10_results:
    top_10_df = pd.concat([top_10_df, pd.DataFrame({'userID': [user_id_to_predict],
                                                     'itemID': [item_id],
                                                     'prediction': [rating]})], ignore_index=True)

print("Top 10 recommended items for user", user_id_to_predict)
print(top_10_df)

Top 10 recommended items for user 1
  userID itemID  prediction
0      1      7    3.959796
1      1     12    3.901928
2      1     43    3.897559
3      1     26    3.888045
4      1     44    3.848835
5      1     21    3.836827
6      1     17    3.787857
7      1     23    3.782312
8      1     24    3.780149
9      1     10    3.759561


In [14]:
# Specify the path to your CSV file
csv_file_path = "/Users/marianareyes/Desktop/ie_tower/chatbots-2/chatlib/datasets/profiles.csv"

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(csv_file_path)

data.head()

,itemID,name,gender,age,year,major,nationality,languages,hobbies
0,1,John Smith,Male,20,2,Computer Science and Artificial Intelligence,USA,English,"Playing video games, reading, hiking"
1,2,Emily Johnson,Female,21,3,Business Administration,Canada,"English, French","Painting, playing guitar, photography"
2,3,Michael Williams,Male,22,4,Economics,UK,English,"Playing football, watching movies, traveling"
3,4,Sarah Brown,Female,20,1,Communication and Digital Media,Australia,English,"Writing, photography, dancing"
4,5,David Jones,Male,19,1,Architecture,Germany,"German, English","Drawing, playing piano, cooking"
